# SimPy Tutorial
https://towardsdatascience.com/introduction-to-simulation-with-simpy-322606d4ba0c

https://simpy.readthedocs.io/en/latest/simpy_intro/basic_concepts.html

In [1]:
import simpy
import pandas as pd
import numpy  as np
from scipy.stats import norm
from scipy.stats import expon

In [2]:
def car(env):
     while True:
         print('Start parking at %d' % env.now)
         parking_duration = 5
         yield env.timeout(parking_duration)
         print('Start driving at %d' % env.now)
         trip_duration = 2
         yield env.timeout(trip_duration)

In [3]:
env = simpy.Environment()
env.process(car(env))

env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


In [4]:
class Car(object):
     def __init__(self, env):
         self.env = env
         # Start the run process everytime an instance is created.
         self.action = env.process(self.run())

     def run(self):
         while True:
             print('Start parking and charging at %d' % self.env.now)
             charge_duration = 5
             # We yield the process that process() returns
             # to wait for it to finish
             yield self.env.process(self.charge(charge_duration))

             # The charge process has finished and
             # we can start driving again.
             print('Start driving at %d' % self.env.now)
             trip_duration = 2
             yield self.env.timeout(trip_duration)

     def charge(self, duration):
         yield self.env.timeout(duration)

In [5]:
env = simpy.Environment()
car = Car(env)
#env.process(car(env))

env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


## SimPy: interrupting processes via interruption handling (interrupted object/class/process)

In [6]:
def driver(env, car):
     yield env.timeout(3)
     car.action.interrupt()

In [7]:
class Car(object):
     def __init__(self, env):
         self.env = env
         # Start the run process everytime an instance is created.
         self.action = env.process(self.run())

     def run(self):
         while True:
             print('Start parking and charging at %d' % self.env.now)
             charge_duration = 5
            # We may get interrupted while charging the battery
             try:
                 yield self.env.process(self.charge(charge_duration))
             except simpy.Interrupt:
                 # When we received an interrupt, we stop charging and
                 # switch to the "driving" state
                 print(f'Was interrupted at {env.now}. Hope, the battery is full enough ...')

             # The charge process has finished and
             # we can start driving again.
             print('Start driving at %d' % self.env.now)
             trip_duration = 2
             yield self.env.timeout(trip_duration)

     def charge(self, duration):
         yield self.env.timeout(duration)

In [8]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))

env.run(until=15)

Start parking and charging at 0
Was interrupted at 3. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


#### Resources
- basic resources: FIFO

In [9]:
def car(env, name, bcs, driving_time, charge_duration):
     """BCS: battery charging station"""
     # Simulate driving to the BCS
     yield env.timeout(driving_time)

     # Request one of its charging spots
     print('%s arriving at %d' % (name, env.now))
     # with statement automatically releases resources
     # otherwise release() needs to be called separately
     with bcs.request() as req:
         yield req

         # Charge the battery
         print('%s starting to charge at %s' % (name, env.now))
         yield env.timeout(charge_duration)
         print('%s leaving the bcs at %s' % (name, env.now))

In [10]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)

In [11]:
for i in range(4):
     env.process(car(env, 'Car %d' % i, bcs, i*2, 5))

In [12]:
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


In [13]:
import random
import simpy

In [14]:
"""
Machine shop example

Covers:

- Interrupts
- Resources: PreemptiveResource

Scenario:
  A workshop has *n* identical machines. A stream of jobs (enough to
  keep the machines busy) arrives. Each machine breaks down
  periodically. Repairs are carried out by one repairman. The repairman
  has other, less important tasks to perform, too. Broken machines
  preempt theses tasks. The repairman continues them when he is done
  with the machine repair. The workshop works continuously.

"""

RANDOM_SEED = 42
PT_MEAN = 10.0         # Avg. processing time in minutes
PT_SIGMA = 2.0         # Sigma of processing time
MTTF = 300.0           # Mean time to failure in minutes
BREAK_MEAN = 1 / MTTF  # Param. for expovariate distribution
REPAIR_TIME = 30.0     # Time it takes to repair a machine in minutes
JOB_DURATION = 30.0    # Duration of other jobs in minutes
NUM_MACHINES = 10      # Number of machines in the machine shop
WEEKS = 4              # Simulation time in weeks
SIM_TIME = WEEKS * 7 * 24 * 60  # Simulation time in minutes


def time_per_part():
    """Return actual processing time for a concrete part."""
    return random.normalvariate(PT_MEAN, PT_SIGMA)


def time_to_failure():
    """Return time until next failure for a machine."""
    return random.expovariate(BREAK_MEAN)


class Machine(object):
    """A machine produces parts and my get broken every now and then.

    If it breaks, it requests a *repairman* and continues the production
    after the it is repaired.

    A machine has a *name* and a numberof *parts_made* thus far.

    """
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False

        # Start "working" and "break_machine" processes for this machine.
        self.process = env.process(self.working(repairman))
        env.process(self.break_machine())

    def working(self, repairman):
        """Produce parts as long as the simulation runs.

        While making a part, the machine may break multiple times.
        Request a repairman when this happens.

        """
        while True:
            # Start making a new part
            done_in = time_per_part()
            while done_in:
                try:
                    # Working on the part
                    start = self.env.now
                    yield self.env.timeout(done_in)
                    done_in = 0  # Set to 0 to exit while loop.

                except simpy.Interrupt:
                    self.broken = True
                    done_in -= self.env.now - start  # How much time left?

                    # Request a repairman. This will preempt its "other_job".
                    with repairman.request(priority=1) as req:
                        yield req
                        yield self.env.timeout(REPAIR_TIME)

                    self.broken = False

            # Part is done.
            self.parts_made += 1

    def break_machine(self):
        """Break the machine every now and then."""
        while True:
            yield self.env.timeout(time_to_failure())
            if not self.broken:
                # Only break the machine if it is currently working.
                self.process.interrupt()


def other_jobs(env, repairman):
    """The repairman's other (unimportant) job."""
    while True:
        # Start a new job
        done_in = JOB_DURATION
        while done_in:
            # Retry the job until it is done.
            # It's priority is lower than that of machine repairs.
            with repairman.request(priority=2) as req:
                yield req
                try:
                    start = env.now
                    yield env.timeout(done_in)
                    done_in = 0
                except simpy.Interrupt:
                    done_in -= env.now - start

In [15]:
# Setup and start the simulation
print('Machine shop')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)
machines = [Machine(env, 'Machine %d' % i, repairman)
            for i in range(NUM_MACHINES)]
env.process(other_jobs(env, repairman))

# Execute!
env.run(until=SIM_TIME)

# Analyis/results
print('Machine shop results after %s weeks' % WEEKS)
for machine in machines:
    print('%s made %d parts.' % (machine.name, machine.parts_made))

Machine shop
Machine shop results after 4 weeks
Machine 0 made 3251 parts.
Machine 1 made 3273 parts.
Machine 2 made 3242 parts.
Machine 3 made 3343 parts.
Machine 4 made 3387 parts.
Machine 5 made 3244 parts.
Machine 6 made 3269 parts.
Machine 7 made 3185 parts.
Machine 8 made 3302 parts.
Machine 9 made 3279 parts.


In [16]:
random.normalvariate()

1.3192211945238288

## JSSP instance

In [17]:
import numpy as np
import numpy.typing as npt
import simpy

In [18]:
def gen_rnd_JSSP_inst(
    n_jobs: int,
    n_machines: int,
    seed: int = 42,
) -> tuple[int, int, int, npt.NDArray[np.uint16], npt.NDArray[np.uint16], npt.NDArray[np.uint16]]:
    """
    Generates random job shop instance with given number of jobs and machines'
    - each job on all machines
    - max processing time = 9
    
    Output:
        - n_jobs: number of jobs
        - n_machines: number of machines
        - n_tasks: number of tasks
        - mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        - mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)
        - mat_OpID: matrix of operation IDs starting by index 1 | shape=(n_jobs,n_machines)
    """
    # generate random process time matrix shape=(n_jobs, n_machines)
    np_rnd_gen = np.random.default_rng(seed=seed)
    mat_ProcTimes = np_rnd_gen.integers(1, 10, size=(n_jobs,n_machines), dtype=np.uint16)
    
    # generate randomly shuffled job machine combinations
    # machine IDs from 1 to n_machines
    temp = np.arange(0, (n_machines), step=1, dtype=np.uint16)
    temp = np.expand_dims(temp, axis=0)
    # repeat dummy line until number n_jobs is reached
    temp = np.repeat(temp, n_jobs, axis=0)
    # randomly permute the machine indices job-wise
    mat_JobMachID = np_rnd_gen.permuted(temp, axis=1)
    
    # generate operation ID matrix
    n_ops = n_jobs * n_machines
    temp2 = np.arange(0, (n_ops), step=1, dtype=np.uint16)
    mat_OpID = temp2.reshape(n_jobs, -1)
    
    return n_jobs, n_machines, n_ops, mat_ProcTimes, mat_JobMachID, mat_OpID

In [19]:
(n_jobs, n_machines, n_ops, 
 mat_ProcTimes, mat_JobMachID, mat_OpID) = gen_rnd_JSSP_inst(2,3)

In [20]:
mat_ProcTimes

array([[2, 1, 9],
       [7, 9, 6]], dtype=uint16)

In [21]:
mat_JobMachID

array([[2, 0, 1],
       [0, 1, 2]], dtype=uint16)

##### Brainstorming of ways to import machine names and IDs
*Case 1: only names are given*
- read machine names and assert IDs to them
- build data structure with name and ID bundled (maybe as property of a machine class)

*Case 2: IDs are given*
- only building data structure with name and ID bundled

*Data Structure:*
- if only mapping of two pairs in each direction (lookup ID or lookup machine name)
    - bi-directional dictionary

In [72]:
import numpy as np
import numpy.typing as npt
import simpy
from typing import TypeAlias
from collections import OrderedDict
import logging
import sys
import pandas as pd
from pandas import DataFrame
from pandas import Series
from functools import lru_cache

# type aliases
SimPyEnv: TypeAlias = simpy.core.Environment
EnvID: TypeAlias = int
JobID: TypeAlias = int
CustomID: TypeAlias = str | int | None
InfstructObj: TypeAlias = object # better naming in future

# forward reference, referenced before assignment
#Job: TypeAlias = 'Job'
#Dispatcher: TypeAlias = 'Dispatcher'

# logging
# IPython compatibility
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#LOGGING_LEVEL = 'DEBUG'
logger = logging.getLogger('base')
#logger.setLevel(LOGGING_LEVEL)

In [23]:
test = set(['test', 'test2', 'test3', 'test'])

In [24]:
test

{'test', 'test2', 'test3'}

- dedicated environment class with information on associated resources and jobs
- maybe add possibility of using subsystems (bundle of resources with unique identifiers)

In [25]:
### OLD LEGACY CODE
class SimulationEnvironment(simpy.core.Environment):
    
    def __init__(
        self,
        *args,
        **kwargs
    ) -> None:
        """
        
        """
        super().__init__(*args, **kwargs)
        
        # resource data base as simple Pandas DataFrame
        self._infstruct_prop: dict[str, type] = {
            'env_id': int,
            'custom_id': object,
            'object': object,
            'name': str,
        }
        self._res_db: DataFrame = pd.DataFrame(columns=list(self._infstruct_prop.keys()))
        self._res_db: DataFrame = self._res_db.astype(self._infstruct_prop)
        self._res_lookup_props: set[str] = set(['env_id', 'custom_id', 'name'])
        
        ############## LEGACY CODE
        ### legacy code, changed approach to tabular data structure
        """
        self.id_counter: EnvID = 0
        self.resources: dict[EnvID, object] = dict()
        self._custom_identifiers: set[CustomID] = set()
        self._custom_from_env_ids: dict[EnvID, CustomID] = dict()
        self._custom_to_env_ids: dict[CustomID, EnvID] = dict()
        """
    
    def register_object(
        self,
        custom_identifier: CustomID,
        obj: InfstructObj,
        name: str | None,
    ) -> tuple[EnvID, str]:
        """
        registers an infrastructure object in the environment by assigning an unique id and 
        adding the object to the associated resources of the environment
        
        object:     env resource
        returns:
            env_id: assigned env ID
        """
        # obtain env_id and set counter up
        env_id = self.id_counter
        self.id_counter += 1
        
        # custom name
        if name is None:
            name = f'M_env_{env_id}'
        
        # new entry for resource data base
        new_entry: DataFrame = pd.DataFrame({
                                'env_id': [env_id],
                                'custom_id': [custom_identifier],
                                'object': [obj],
                                'name': [name]})
        new_entry: DataFrame = new_entry.astype(self._infstruct_prop)
        self._res_db = pd.concat([self._res_db, new_entry], ignore_index=True)
        
        ############### OLD
        self.resources[env_id] = obj
        logger.info(f"Successfully registered object with EnvID {env_id} and name {name}")
        
        return env_id, name
    
    def get_resource_db(self) -> DataFrame:
        """
        obtain a current overview of registered objects in the environment
        """
        return self._res_db

    def get_res_obj_by_prop(
        self,
        property: str, 
        val: EnvID | CustomID | str,
    ) -> InfstructObj:
        """
        obtain a resource object from the environment by its property and corresponding value
        properties: env_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._res_lookup_props:
            raise KeyError(f"Property '{property}' is not allowed. Choose from {self._res_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise ValueError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        temp1: Series = self._res_db.loc[self._res_db[property] == val, 'object']
        # check for empty search result, at least one result necessary
        if len(temp1) == 0:
            raise KeyError(f"There were no resources found for the property '{property}' \
                            with the value '{val}'")
        # check for multiple entries with same prop-value pair
        ########### PERHAPS CHANGE NECESSARY
        ### multiple entries but only one returned --> prone to errors
        elif len(temp1) > 1:
            # warn user
            logger.warning(f"CAUTION: There are multiple resources which share the \
                        same value '{val}' for the property '{property}'. \
                        Only the first entry is returned.")
        
        return temp1.at[0]
    
    
    ############## LEGACY CODE
    ### legacy code, changed approach to tabular data structure
    ############# NECESSARY???
    ### new way of data table with all registered infrastructure objects
    ### custom identifier for user interface to transfer object information
    ### out of the model itself (together with object's name)
    ### duplicate check convenient, but not necessary
    ### uniqeness of env_ids is guaranteed by the inner logic

    def register_custom_identifier(
        self,
        env_ID: EnvID,
        custom_identifier: CustomID,
    ) -> bool:
        """
        maps custom identifiers of resources to env_id and vice versa
        """
        self._custom_from_env_ids[env_ID] = custom_identifier
        # check if custom identifier is a duplicate
        if custom_identifier not in self._custom_identifiers:
            self._custom_identifiers.add(custom_identifier)
            self._custom_to_env_ids[custom_identifier] = env_ID
        elif self._custom_to_env_ids[custom_identifier] == env_ID:
            logger.info("Custom identifier already associated with given environment ID.")
        # ambigious, custom_identifier is a duplicate
        else:
            logger.warning("The custom identifier is ambigous and was already used before for another environment ID. \
                This object can only be uniquely identified by its environment ID, not by the custom identifier provided.")
        
        return True
    

    
    def get_custom_from_env_id(
        self,
        env_ID: EnvID,
    ) -> CustomID:
        """
        #get custom ID by an object's environment id
        """
        try:
            return self._custom_from_env_ids[env_ID]
        except KeyError as error:
            logger.error(f"The provided key {error} does not exist!")
            raise
    
    def get_custom_to_env_id(
        self,
        custom_identifier: CustomID,
    ) -> EnvID:
        """
        #get environment ID by an object's custom identifier
        """
        try:
            return self._custom_to_env_ids[custom_identifier]
        except KeyError as error:
            logger.error(f"The provided key {error} does not exist!")
            raise
        
        

In [153]:
class SimulationEnvironment(simpy.core.Environment):
    Dispatcher: TypeAlias = 'Dispatcher'
    
    def __init__(
        self,
        *args,
        **kwargs,
    ) -> None:
        """
        
        """
        super().__init__(*args, **kwargs)
        
        # resource data base as simple Pandas DataFrame
        self._infstruct_prop: dict[str, type] = {
            'env_id': int,
            'custom_id': object,
            'resource': object,
            'name': str,
            'res_type': str,
        }
        self._res_db: DataFrame = pd.DataFrame(columns=list(self._infstruct_prop.keys()))
        self._res_db: DataFrame = self._res_db.astype(self._infstruct_prop)
        self._res_lookup_props: set[str] = set(['env_id', 'custom_id', 'name'])
        
        # env identifiers
        self.id_counter: EnvID = 0
        
        # job dispatcher
        self._dispatcher_registered: bool = False
        self._dispatcher: Dispatcher = None
        
        ############## LEGACY CODE
        ### legacy code, changed approach to tabular data structure
        """
        self.id_counter: EnvID = 0
        self.resources: dict[EnvID, object] = dict()
        self._custom_identifiers: set[CustomID] = set()
        self._custom_from_env_ids: dict[EnvID, CustomID] = dict()
        self._custom_to_env_ids: dict[CustomID, EnvID] = dict()
        """
        
    def _obtain_env_id(self) -> EnvID:
        """Simple counter function for managing environment IDs"""
        # assign env_id and set counter up
        env_id = self.id_counter
        self.id_counter += 1
        
        return env_id
    
    def register_dispatcher(
        self,
        dispatcher: Dispatcher,
    ) -> EnvID:
        """
        Registers a dispatcher instance for the environment. Only one instance per environment is allowed.
        returns: EnvID for the dispatcher instance
        """
        # obtain env_id
        env_id = self._obtain_env_id()
        
        if not self._dispatcher_registered:
            self._dispatcher = dispatcher
            self._dispatcher_registered = True
        else:
            raise AssertionError("There is already a registered dispatcher instance \
                                 Only one instance per environement is allowed.")
        
        return True
    
    @property
    def dispatcher(self) -> Dispatcher:
        """obtain the current registered dispatcher instance of the environment"""
        return self._dispatcher
    
    def register_resource(
        self,
        custom_identifier: CustomID,
        obj: InfstructObj,
        name: str | None,
    ) -> tuple[EnvID, str]:
        """
        registers an infrastructure object in the environment by assigning an unique id and 
        adding the object to the associated resources of the environment
        
        object:     env resource
        returns:
            env_id: assigned env ID
        """
        # obtain env_id
        env_id = self._obtain_env_id()
        
        # custom name
        if name is None:
            name = f'M_env_{env_id}'
        
        # new entry for resource data base
        new_entry: DataFrame = pd.DataFrame({
                                'env_id': [env_id],
                                'custom_id': [custom_identifier],
                                'resource': [obj],
                                'name': [name],
                                'res_type': [obj.res_type]})
        new_entry: DataFrame = new_entry.astype(self._infstruct_prop)
        self._res_db = pd.concat([self._res_db, new_entry], ignore_index=True)
        
        logger.info(f"Successfully registered object with EnvID {env_id} and name {name}")
        
        return env_id, name
    
    @property
    def res_db(self) -> DataFrame:
        """obtain a current overview of registered objects in the environment"""
        return self._res_db

    @lru_cache(maxsize=200)
    def get_res_obj_by_prop(
        self,
        property: str, 
        val: EnvID | CustomID | str,
        target_prop: str = 'resource',
    ) -> InfstructObj:
        """
        obtain a resource object from the environment by its property and corresponding value
        properties: env_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._res_lookup_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._res_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        temp1: Series = self._res_db.loc[self._res_db[property] == val, target_prop]
        # check for empty search result, at least one result necessary
        if len(temp1) == 0:
            raise IndexError(f"There were no resources found for the property '{property}' \
                            with the value '{val}'")
        # check for multiple entries with same prop-value pair
        ########### PERHAPS CHANGE NECESSARY
        ### multiple entries but only one returned --> prone to errors
        elif len(temp1) > 1:
            # warn user
            logger.warning(f"CAUTION: There are multiple resources which share the \
                        same value '{val}' for the property '{property}'. \
                        Only the first entry is returned.")
        
        return temp1.iat[0]
    
        

In [144]:
class Machine(simpy.resources.resource.Resource):
    
    def __init__(
        self,
        env: SimulationEnvironment,
        custom_identifier: CustomID = None,
        name: str | None = None,
        num_slots: int = 1,
    ) -> None:
        """
        env:        SimPy Environment in which machine is embedded
        num_slots:  capacity of the machine, if multiple processing 
                    slots available at the same time > 1, default=1
        """
        # intialize base class
        super().__init__(env=env, capacity=num_slots)
        
        ############# custom identifiers only over env_id
        ### associate env_id with custom_id in env
        ### lookup env_id of object in environment and obtain custom_id
        
        
        ############# CHECK IF NECESSARY IN FUTURE
        """
        if custom_identifier is not None:
            ret = env.register_custom_identifier(
                    env_ID=self.env_id, custom_identifier=custom_identifier)
        """
        
        
        # assert machine information and register object in the environment
        self.env = env
        self.custom_identifier = custom_identifier
        self.res_type: str = 'Machine'
        self.env_id, self.name = env.register_resource(
                                custom_identifier=self.custom_identifier,
                                obj=self, name=name)
        
        
        # currently processed job
        self.current_job_ID: int | None = None
        self.current_job: Job | None = None
        
        # machine state parameters
        self.is_occupied: bool = False
        self.is_waiting: bool = False
        self.is_blocked: bool = False
        self.is_failed: bool = False
        # maybe for future, curently no working time calendars planned
        self.is_paused: bool = False
        
        # time in state parameters
        self.time_occupied: float = 0.
        self.time_waiting: float = 0.
        self.time_blocked: float = 0.
        self.time_failed: float = 0.
        
        # number of inputs/outputs
        self.num_jobs_input: int = 0
        self.num_jobs_output: int = 0

In [145]:
(n_jobs, n_machines, n_ops, 
 mat_ProcTimes, mat_JobMachID, mat_OpID) = gen_rnd_JSSP_inst(2,3)

In [146]:
mat_ProcTimes

array([[2, 1, 9],
       [7, 9, 6]], dtype=uint16)

In [147]:
mat_JobMachID

array([[2, 0, 1],
       [0, 1, 2]], dtype=uint16)

In [148]:
mat_OpID

array([[0, 1, 2],
       [3, 4, 5]], dtype=uint16)

In [149]:
env = SimulationEnvironment()

for machine in np.unique(mat_JobMachID):
    MachInst = Machine(env=env, custom_identifier=machine)

INFO:base:Successfully registered object with EnvID 0 and name M_env_0
INFO:base:Successfully registered object with EnvID 1 and name M_env_1
INFO:base:Successfully registered object with EnvID 2 and name M_env_2


In [150]:
res_db = env.res_db
res_db

,env_id,custom_id,resource,name,res_type
0,0,0,<__main__.Machine object at 0x0000025C679D22D0>,M_env_0,Machine
1,1,1,<__main__.Machine object at 0x0000025C680E0090>,M_env_1,Machine
2,2,2,<__main__.Machine object at 0x0000025C680C14D0>,M_env_2,Machine


In [151]:
ret = env.get_res_obj_by_prop(property='env_id', val=2)

In [152]:
ret

- job sets as own class currently not necessary, use standard OrderedDict instead

In [35]:
class JobSet(object):
    
    def __init__(
        self,
        mat_ProcTimes: npt.NDArray[np.uint16],
        mat_JobMachID: npt.NDArray[np.uint16],
        mat_OpID: npt.NDArray[np.uint16],
    ):
        """
        mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)
        mat_OpID: matrix of operation IDs starting by index 1 | shape=(n_jobs,n_machines)
        """
        
        self._jobs = OrderedDict()
        
        for job_id in range(len(mat_ProcTimes)):
            temp1 = mat_ProcTimes[job_id].tolist()
            temp2 = mat_JobMachID[job_id].tolist()
            temp3 = mat_OpID[job_id].tolist()
            job = Job(
                identifier=job_id,
                proc_times=temp1,
                machine_order=temp2,
                operation_identifiers=temp3,
            )
            self._jobs[job_id] = job
            
    def __getitem__(
        self,
        job_id: int,
    ):
        return self._jobs[job_id]
    

---

#### **Adding database approach also to the job dispatcher**
**properties**:
- job_id
- custom_id
- job instance
- name
- product_type (for later implementation of different product types)

In [155]:
class Dispatcher(object):
    Job: TypeAlias = 'Job'
    Dispatcher: TypeAlias = 'Dispatcher'
    
    def __init__(
        self,
        env: SimulationEnvironment = None,
    ) -> None:
        """
        Dispatcher class for given environment (only one dispatcher for each environment)
        - different functions to monitor all jobs in the environment
        - jobs report back their states to the JobWatcher
        """
                
        # job data base as simple Pandas DataFrame
        self._infstruct_prop: dict[str, type] = {
            'job_id': int,
            'custom_id': object,
            'job': object,
            'name': str,
            'job_type': str,
        }
        self._job_db: DataFrame = pd.DataFrame(columns=list(self._infstruct_prop.keys()))
        self._job_db: DataFrame = self._job_db.astype(self._infstruct_prop)
        self._job_lookup_props: set[str] = set(['job_id', 'custom_id', 'name'])
        
        self.disposable_jobs: dict[int, Job] = dict()
        self.job_pool: OrderedDict[JobID, Job] = OrderedDict()
        self.id_counter: JobID = 0
    
    def gen_job_pool_generic(
        self,
        mat_ProcTimes: npt.NDArray[np.uint16],
        mat_JobMachID: npt.NDArray[np.uint16],
        mat_OpID: npt.NDArray[np.uint16],
    ) -> OrderedDict[JobID, Job]:
        """
        function to build a integrated job pool if generic JxM JSSP instances are used
        mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)
        mat_OpID: matrix of operation IDs starting by index 1 | shape=(n_jobs,n_machines)
        """
            
        for job_id in range(len(mat_ProcTimes)):
            temp1 = mat_ProcTimes[job_id].tolist()
            temp2 = mat_JobMachID[job_id].tolist()
            temp3 = mat_OpID[job_id].tolist()
            JobInst = Job(
                identifier=job_id,
                proc_times=temp1,
                machine_order=temp2,
                operation_identifiers=temp3,
                dispatcher=self,
            )
            ######### ADD TO JOB (bottom-up approach) #######################
            ### jobs add themselves to job pool
            self.job_pool[job_id] = JobInst
            
        return self.job_pool
     
    @property
    def job_db(self) -> DataFrame:
        """
        obtain a current overview of registered objects in the environment
        """
        return self._job_db

    @lru_cache(maxsize=200)
    def get_job_obj_by_prop(
        self,
        property: str, 
        val: EnvID | CustomID | str,
        target_prop: str = 'job',
    ) -> InfstructObj:
        """
        obtain a job object from the dispatcher by its property and corresponding value
        properties: job_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._job_lookup_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._job_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        temp1: Series = self._job_db.loc[self._job_db[property] == val, target_prop]
        # check for empty search result, at least one result necessary
        if len(temp1) == 0:
            raise IndexError(f"There were no resources found for the property '{property}' \
                            with the value '{val}'")
        # check for multiple entries with same prop-value pair
        ########### PERHAPS CHANGE NECESSARY
        ### multiple entries but only one returned --> prone to errors
        elif len(temp1) > 1:
            # warn user
            logger.warning(f"CAUTION: There are multiple resources which share the \
                        same value '{val}' for the property '{property}'. \
                        Only the first entry is returned.")
        
        return temp1.iat[0]
    
    ################# REWORK ##################
    def get_disposable_jobs(
        self,
        job_set: OrderedDict,
    ) -> tuple[list[JobID], list[Job]]:
        """
        function needs to be reworked, jobs should report back information to a dispatcher instance
        (bottom-up instead of top-down)
        """
        #########################################
        self._disposable_jobs_ID: list[int] = list()
        self._disposable_jobs: list[Job] = list()
        
        for job_id, job in job_set.items():
            if job.is_disposable:
                self._disposable_jobs_ID.append(job_id)
                self._disposable_jobs.append(job)
                
        return self._disposable_jobs_ID, self._disposable_jobs

### Logic behind simulation model approach
*Model of Resource and Load*:
- system consists of physical objects, also called infrastructure
    - each element can be considered as encapsulated resource
- stress can be put on each system by occupying resources, also called load
    - definition of load depends on system type and modelling, e.g. production jobs for production systems or customers for cashiers in a shop

*Guiding Priciples*:
- **load objects** can only be spatially and temporally modified by **resources**
    - whole **routing logic is implemented in the resources**: no load object can change its state without a associated resource
    - load objects **contain the necessary information** which is essential for their further processing

In [37]:
class Operation(object):
    
    def __init__(
        self,
        identifier: int,
        proc_times: float,
        machine_identifier: int,
        name: str | None = None,
    ) -> None:
        """
        identifier:         operation's ID
        proc_times:         operation's processing times
        machine_identifier: ID of machine on which operation is processed
        """
        # !!!!!!!!! perhaps processing times in future multiple entries depending on associated machine
        # change of input format necessary, currently only one machine for each operation
        # no groups, no differing processing times for different machines 

        # assert operation information
        self.identifier = identifier
        # custom name
        if name is not None:
            self.name = name
        else:
            self.name = f'O{identifier}'
            
        # process information
        self.proc_time = proc_times
        ########### adding machine instances
        ### perhaps adding machine sets if multiple machines possible (machine groups)
        self.target_machine = machine_identifier

In [38]:
class Job(object):
    Job: TypeAlias = 'Job'
    Dispatcher: TypeAlias = 'Dispatcher'
    
    def __init__(
        self,
        identifier: JobID,
        proc_times: list[float],
        machine_order: list[int],
        operation_identifiers: list[int],
        dispatcher: Dispatcher,
        name: str | None = None,
    ) -> None:
        """
        identifier:             job's ID
        proc_times:             list of processing times for each operation
        machine_order:          list of machine IDs
        operation_identifiers:  list of operation IDs
        """
        # intialize base class
        super().__init__()
        
        ### BASIC INFORMATION ###
        # assert job information
        self.identifier = identifier
        # custom name
        if name is not None:
            self.name = name
        else:
            self.name = f'J{identifier}'
        self.Dispatcher = dispatcher
        
        
        ### OPERATIONS ##
        self.operations = list()
        
        for idx, op_ID in enumerate(operation_identifiers):
            Op = Operation(
                identifier=op_ID,
                proc_times=proc_times[idx],
                machine_identifier=machine_order[idx],
            )
            self.operations.append(Op)
            
        self.total_num_ops: int = len(self.operations)
        self.num_finished_ops: int = 0
        self.current_op: Operation = self.operations[0]
        
        ### STATE ###
        # intra-process job state parameters
        # job is being processed, maybe better naming in future
        self.is_occupied: bool = False
        # waiting state only when released
        self.is_waiting: bool = False
        # if lying on failed machine
        self.is_failed: bool = False
        
        # intra-process time characteristics
        self.time_occupied: float = 0.
        self.time_waiting: float = 0.
        self.time_failed: float = 0.
        
        # inter-process job state parameters
        # first operation scheduled --> released job
        self.is_released: bool = False
        # job's next operation is disposable
        # true for each new job, maybe reworked in future for jobs with
        # a start date later than creation date
        self.is_disposable: bool = True
        self.Dispatcher.disposable_jobs[self.identifier] = self
        # last operation ended --> finished job
        self.is_finished: bool = False
        
        # inter-process time characteristics
        # time of first operation starting point
        self.time_entry: float = 0.
        # time of last operation ending point
        self.time_exit: float = 0.
        
        # current resource location
        self.current_resource: object | None = None # specify type if class definition finished
        
        
    def obtain_current_op(self) -> Operation:
        """
        returns the current operation of the job
        If a job is currently being processed its current operation is 
        not changed until this operation is finished.
        """
        return self.current_op

In [39]:
(n_jobs, n_machines, n_ops, 
 mat_ProcTimes, mat_JobMachID, mat_OpID) = gen_rnd_JSSP_inst(2,3)

In [40]:
mat_ProcTimes

array([[2, 1, 9],
       [7, 9, 6]], dtype=uint16)

In [41]:
mat_JobMachID

array([[2, 0, 1],
       [0, 1, 2]], dtype=uint16)

In [42]:
mat_OpID

array([[0, 1, 2],
       [3, 4, 5]], dtype=uint16)

- job sets as own class currently not necessary, use standard OrderedDict instead

In [43]:
class JobSet(object):
    
    def __init__(
        self,
        mat_ProcTimes: npt.NDArray[np.uint16],
        mat_JobMachID: npt.NDArray[np.uint16],
        mat_OpID: npt.NDArray[np.uint16],
    ):
        """
        mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)
        mat_OpID: matrix of operation IDs starting by index 1 | shape=(n_jobs,n_machines)
        """
        
        self._jobs = OrderedDict()
        
        for job_id in range(len(mat_ProcTimes)):
            temp1 = mat_ProcTimes[job_id].tolist()
            temp2 = mat_JobMachID[job_id].tolist()
            temp3 = mat_OpID[job_id].tolist()
            job = Job(
                identifier=job_id,
                proc_times=temp1,
                machine_order=temp2,
                operation_identifiers=temp3,
            )
            self._jobs[job_id] = job
            
    def __getitem__(
        self,
        job_id: int,
    ):
        return self._jobs[job_id]
    

In [44]:
machine_infrastructure = OrderedDict()
env = SimulationEnvironment()

for machine in np.unique(mat_JobMachID):
    MachInst = Machine(env)

INFO:base:Successfully registered object with EnvID 0 and name M_env_0
INFO:base:Successfully registered object with EnvID 1 and name M_env_1
INFO:base:Successfully registered object with EnvID 2 and name M_env_2


In [45]:
MachInst.env_id

2

In [46]:
DispatcherInst = Dispatcher()

In [47]:
mat_ProcTimes

array([[2, 1, 9],
       [7, 9, 6]], dtype=uint16)

In [48]:
job_pool = DispatcherInst.gen_job_pool_generic(
    mat_ProcTimes=mat_ProcTimes,
    mat_JobMachID=mat_JobMachID,
    mat_OpID=mat_OpID,
)

In [49]:
test = DispatcherInst.disposable_jobs

In [50]:
test

{0: <__main__.Job at 0x25c678b3450>, 1: <__main__.Job at 0x25c678bc290>}

In [51]:
import salabim as sim
import simpy
import random

In [52]:
random.seed(42)

In [53]:
def execution_time():
    return random.normalvariate(mu=10., sigma=1.5)

In [54]:
def test_process_put(env, store):
    TIME = 10
    counter = 10
    while counter:
        item = yield store.get()
        print(f"I got item {item} at {env.now}")
        yield env.timeout(TIME)
        #yield env.timeout(TIME)
        #print(f"I got executed at {env.now}")
        counter -= 1
        
def placer_process(env, store):
    TIME = 2
    counter = 6
    while counter:
        yield env.timeout(TIME)
        yield store.put(counter)
        print(f"Placed object {counter} in store")
        counter -= 1
        
def placer_process_machine(env, machine):
    TIME = 2
    counter = 6
    PROCESSING_TIME = 10
    while counter:
        #yield env.timeout(TIME)
        with machine.request() as req:
            yield req
        yield env.timeout(PROCESSING_TIME)
        #yield machine.put(counter)
        print(f"Item processed on machine {machine} at {env.now}")
        counter -= 1

In [55]:
env = simpy.Environment()
store = simpy.Store(env=env)
store.put(1)
machine = simpy.resources.resource.Resource(env=env)
#env.process(test_process_put(env=env, store=store))
#env.process(placer_process(env=env, store=store))
env.process(placer_process_machine(env=env, machine=machine))


<Process(placer_process_machine) object at 0x25c67526b90>

In [56]:
env.run()

Item processed on machine <simpy.resources.resource.Resource object at 0x0000025C67896050> at 10
Item processed on machine <simpy.resources.resource.Resource object at 0x0000025C67896050> at 20
Item processed on machine <simpy.resources.resource.Resource object at 0x0000025C67896050> at 30
Item processed on machine <simpy.resources.resource.Resource object at 0x0000025C67896050> at 40
Item processed on machine <simpy.resources.resource.Resource object at 0x0000025C67896050> at 50
Item processed on machine <simpy.resources.resource.Resource object at 0x0000025C67896050> at 60
